Sequential:

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu")
])

#possiamo aggiungere successivamente (e nominare) un layer
model.add(layers.Dense(10, activation="softmax", name="softmax_layer"))
model.summary()


2025-10-08 21:18:45.402566: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_layer (Dense)           │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Functional:

In [2]:
inputs = keras.Input(shape=(3,), name="input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

2025-10-08 21:27:20.919556: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)

Un sistema per catalogare i ticket del support clienti per priorità e mandarli al dipartimento corretto, modello compilato usando la funzionale:
il modello prende 3 input:
-il titolo del ticket;
-il corpo del ticket;
-eventuali tag aggiunti dall'utente.

In [5]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")
features = layers.Concatenate()([title, text_body, tags])
features = layers.Dense(64, activation="relu")(features)
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
 num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags],
 outputs=[priority, department])

In [6]:
import numpy as np

num_samples = 1280
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(optimizer="rmsprop",
 loss=["mean_squared_error", "categorical_crossentropy"],
 metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit([title_data, text_body_data, tags_data],
 [priority_data, department_data],
 epochs=1)

model.evaluate([title_data, text_body_data, tags_data],
 [priority_data, department_data])

priority_preds, department_preds = model.predict(
 [title_data, text_body_data, tags_data])

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - department_accuracy: 0.3922 - department_loss: 113.5513 - loss: 113.8634 - priority_loss: 0.3121 - priority_mean_absolute_error: 0.4796
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - department_accuracy: 0.1531 - department_loss: 134.8231 - loss: 135.1526 - priority_loss: 0.3295 - priority_mean_absolute_error: 0.4982
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [7]:
model.compile(optimizer="rmsprop",
 loss={"priority": "mean_squared_error", "department":
 "categorical_crossentropy"},
 metrics={"priority": ["mean_absolute_error"], "department":
 ["accuracy"]})

model.fit({"title": title_data, "text_body": text_body_data,
 "tags": tags_data},
 {"priority": priority_data, "department": department_data},
 epochs=1)

model.evaluate({"title": title_data, "text_body": text_body_data,
 "tags": tags_data},
 {"priority": priority_data, "department": department_data})
priority_preds, department_preds = model.predict(
 {"title": title_data, "text_body": text_body_data, "tags": tags_data})

40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - department_accuracy: 0.2656 - department_loss: 86.5029 - loss: 86.8324 - priority_loss: 0.3295 - priority_mean_absolute_error: 0.4982  
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - department_accuracy: 0.0594 - department_loss: 35.4704 - loss: 35.7999 - priority_loss: 0.3295 - priority_mean_absolute_error: 0.4982
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [1]:
from tensorflow import keras
from tensorflow.keras import layers

class CustomerTicketModel(keras.Model):

    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(num_departments, activation="softmax")

    def call(self, intputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

2025-10-09 19:21:35.751009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model = CustomerTicketModel(num_departments=4)
priority, department = model(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

model.compile(optimizer="rmsprop", loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          [priority_data, department_data], epochs=1)

model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data}, 
               [priority_data, department_data])

priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})